# Modelo Hull-White

Veamos la construcción de la función Theta, la simulación y el cálculo de los factores de descuento con las fórmulas del modelo.

In [1]:
# Sirve para importar los datos de una curva
import various_functions_2 as vf

In [2]:
# Obtenemos los valores de la curva
curva = vf.get_curva('./curva_3.csv')
print("Plazos  Factores")
print("----------------")
for x in zip(curva[0], curva[1]):
    print("{0:.4f}: {1:.6%}".format(x[0], x[1]))

Plazos  Factores
----------------
0.0027: 99.997500%
0.0192: 99.982357%
0.0384: 99.964717%
0.0603: 99.945091%
0.0877: 99.920153%
0.1671: 99.847310%
0.2521: 99.763380%
0.3370: 99.674091%
0.4192: 99.584806%
0.5096: 99.485989%
0.7589: 99.180011%
1.0000: 98.856383%
1.5068: 98.118342%
2.0055: 97.269353%
3.0027: 95.419929%
4.0027: 93.449345%
5.0027: 91.416526%
7.0110: 85.173523%
10.0055: 78.181848%
12.0082: 73.680354%
15.0110: 67.415769%
20.0137: 58.246209%
25.0192: 50.669342%
30.0247: 44.232248%
40.0274: 34.143186%
50.0329: 26.709198%


In [3]:
# La curva está expresada en df. La pasamos a tasa.
import math
from bokeh.plotting import figure, show, output_notebook

tenors = []
rates = []
for x in zip(curva[0], curva[1]):
    # df = exp(-rate * yf)
    # log(df) = - rate * yf
    # - log(df) / yf = rate
    
    # yf = x[0] / 365
    rate = - math.log(x[1]) / x[0]
    tenors.append(x[0])
    rates.append(rate)

# Graficamos la curva
def plot_curve(tenors, rates, title):
    # se define la data
    x = tenors
    y = rates

    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title=title, x_axis_label='Tiempo', y_axis_label='Tasa')

    # add a line renderer with legend and line thickness
    p.line(x, y, legend="Curva", line_width=2)

    # show the results
    show(p)
plot_curve(tenors, rates, "Curva Cupón Cero")


Loading BokehJS ...

Recordemos la expresión para Theta:

La fórmula para la función $\theta_{t}$ es:

$$\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\big(1-\exp(-2\gamma^*t)\big)$$

Necesitamos una representación en tiempo contínuo de la curva para poder calcular $f(0,t)$ y su derivada. Para eso, vamos a utilizar un **cubic-spline.**

In [4]:
# Definimos la curva como un spline cúbico
from scipy.interpolate import interp1d
zrate = interp1d(tenors, rates, 'cubic', fill_value="extrapolate")

In [5]:
# Veamos un gráfico
rates_spline = []
tenors_spline = []
steps_per_year = 264.0
dt = 1 / steps_per_year
years = 20
for i in range(0, int(steps_per_year * years)):
    tenors_spline.append(i * dt)
    rates_spline.append(zrate(i * dt))
plot_curve(tenors_spline, rates_spline, "Curva Cupón Cero")

Loading BokehJS ...

In [24]:
# Definimos la derivada primera y segunda como diferencias finitas
def dzrate(t):
    delta = .00000001
    return (zrate(t + delta / 2) - zrate(t - delta / 2)) / (2 * delta)

def d2zrate(t):
    delta = .0001
    return (zrate(t + delta) - 2 * zrate(t) + zrate(t - delta)) / (delta**2)

t = curva[0][1]
print("t: {0:.4f}".format(t))
print("zrate(t): {0:.4%}".format(zrate(t)))
print("dzrate(t): {0:.4%}".format(dzrate(t)))
print("d2zrate(t): {0:.4%}".format(d2zrate(t)))

t: 0.0192
zrate(t): 0.9200%
dzrate(t): 0.1332%
d2zrate(t): -25.3926%


Sabemos que:

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$,

y también que:

$$\frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$$

In [25]:
# Podemos ahora definir las funciones fwd(0,t) y dfwd(0,t)
def fwd(t):
    return zrate(t) + t * dzrate(t)

def dfwd(t):
    return 2 * dzrate(t) + t * d2zrate(t)

print("fwd(t): {0:.4%}".format(fwd(t)))
print("dfwd(t): {0:.4%}".format(dfwd(t)))

fwd(t): 0.9226%
dfwd(t): -0.2205%


In [26]:
# Con esto, podemos ya escribir una expresión para la función Theta.
# Definamos un valor para sigma y gamma
sigma = .015
gamma = .5
r0 = rates_spline[0]

def theta(t):
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux
print("Theta(t): {0:.4%}".format(theta(t)))

Theta(t): 0.2412%


$$\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\big(1-\exp(-2\gamma^*t)\big)$$

In [27]:
tiempos = []
thetas = []
dt = 1 / 264.0
for i in range(0, 66):
    t = i * dt
    tiempos.append(t)
    thetas.append(theta(t))
    
plot_curve(tiempos, thetas, "Valores Theta")

Loading BokehJS ...

Finalmente, podemos simular.

In [28]:
from numpy import random as rnd
def sim_hw(gamma, sigma, theta, r0, num_steps, seed = None):
    r = r0
    dt = 1 / 264.0
    sqdt = math.sqrt(dt)
    sim = [r0]
    rnd.seed(seed)
    for i in range(1, num_steps):
        # print(str(r))
        epsilon = rnd.normal()
        r = r + (theta((i - 1) * dt) - gamma * r) * dt + sigma * sqdt * epsilon
        sim.append(r)
        # print(str(theta((i - 1) * dt)) + "\t" + str(epsilon) + "\t" + str(r) )
    return sim

In [29]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    dt = 1 / 264.0
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)
    # for i in range(0, 5000):
    #    rnd.normal()
    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    for i in range(0, num_steps):
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            # print(str(r))
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
            # print(str(theta_array[j - 1]) + "\t" + str(alea[i][j - 1]) + "\t" + str(r) )
    return sim

In [30]:
def plot_simulation(dt, sim):
    # se define el eje tiempo
    tiempo = [0]
    for i in range(1, len(sim)):
        tiempo.append(i * dt)

    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title="Simulación", x_axis_label='Tiempo', y_axis_label='Tasa')

    # add a line renderer with legend and line thickness
    p.line(tiempo, sim, legend="Curva", line_width=2)

    # show the results
    show(p)

In [31]:
dt = 1 / 264.0
sim = sim_hw(gamma, sigma, theta, r0, 264, 1001)
plot_simulation(dt, sim)

Loading BokehJS ...

In [32]:
print("Última tasa: {0:.4%}".format(sim[263]))

Última tasa: 1.1581%


Se corre una simulación obtenemos r(t) para muchos t.
Lo que realmente queremos es Z(t, T) para todo T.

In [33]:
# Coeficiente B del modelo de HW
def b_hw(gamma, t, T):
    """
    gamma : para 
    sigma: 
    t : plazo de....
    T: plazo ....
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [34]:
# Coeficiente A del modelo de HW
def a_hw(zrate, fwd, gamma, sigma, t, T, verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [35]:
# Factor de descuento según HW
print("sigma: {0:.4%}".format(sigma))
print("gamma: {0:.4%}".format((gamma)))
def zero_hw(r, gamma, sigma, zrate, fwd, t, T):
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

sigma: 1.5000%
gamma: 50.0000%


In [37]:
# Verifiquemos los valores que entrega la fórmula de HW con los datos de la curva
r0 = zrate(1 / 3650000)
print("r0: {0:.4%}".format(r0))
t = 0
T = curva[0][15]
z = zero_hw(r0, gamma, sigma, zrate, fwd, t, T)
print("zero_hw: {0:.8%}".format(round(z, 8)))
print("zero curva:  {0:.8%}".format(round(curva[1][15], 8)))

r0: 0.9107%
zero_hw: 93.44934400%
zero curva:  93.44934500%


Vamos a valorizar, en 1Y más, un bono semestral a tasa fija del 5% que en 1Y más le quedarán 4Y. Podemos además hacer el ejercicio de verificar la simulación calculando los valores de los bonos cero cupón usando la simulación.

In [38]:
first_coupon = 1.5
tasa = .05
num_cupones = 8
bf = vf.bono_tasa_fija(first_coupon, num_cupones, tasa)
bf

([1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
 [0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 1.025])

In [39]:
r1Y = sim[263]
print("r1Y: {0:.4%}".format(r1Y))

r1Y: 1.1581%


In [40]:
# ¿Cuál es la curva en t = 1Y en esta simulación?
t = 1
zeros_1Y = []
for plazo in bf[0]:
    zeros_1Y.append(zero_hw(r1Y, gamma, sigma, zrate, fwd, t, plazo))
for zero in zeros_1Y:
    print(zero)

0.993118942193052
0.984865858813832
0.9758985286005476
0.9665431598716021
0.9567362023692675
0.9468346418614195
0.9371232179213228
0.9263941326085645


In [41]:
# ¿Cuál es el valor presente?
import numpy as np
vp = np.dot(zeros_1Y, bf[1])
print("vp: " + str(vp))

vp: 1.1185819997145547


In [42]:
# Chequear que dot haga lo que creemos que está haciendo
vp1 = 0
for y in zip(bf[1], zeros_1Y):
    vp1 += y[0] * y[1]
vp1

1.1185819997145547

In [45]:
num_sim = 10
def sim_many(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf):
    result = []
    seeds = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
    # seeds = [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
    for i in range(0, num_sim):
        r1y = sim_hw(gamma, sigma, theta, r0, 264, seeds[i])[263]
        zeros_1Y = []
        for plazo in bf[0]:
            zeros_1Y.append(zero_hw(r1y, gamma, sigma, zrate, fwd, t, plazo))
        vp = np.dot(zeros_1Y, bf
                    [1])
        result.append(vp)
    return result

res = sim_many(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf)
res

[1.1539602637600148,
 1.0981495693176266,
 1.1221366349309245,
 1.1308795544336732,
 1.118283304985973,
 1.084574308828541,
 1.1045705835316415,
 1.1275419530348034,
 1.1332979587757626,
 1.1275577827928205]

In [ ]:
def payoff(precio, strike):
    if precio > strike:
        return precio - strike
    else:
        return 0.0

In [ ]:
strike = 1.12
prom = 0.0
for r in res:
    prom += payoff(r, strike)
    print(payoff(r, strike))
e = prom / 10.0
print("E: " + str(prom / 10.0))
v = math.exp(-zrate(1.0) * 1.0) * e
print("V: " + str(v))

exp(-r01*Dt - r12*Dt- r23*Dt - r34*Dt) = exp(-r01*Dt)*exp(-r12*Dt)*exp(-r23*Dt)*exp(-r34*Dt) = df(0, 4) en la simulación 

In [46]:
num_sim = 1000
# dfs = 0
t = .25
sim = 0
seed = None
num_steps = int(t*264)
print("num_steps: " + str(num_steps))
for i in range(0, num_sim):
    sim += math.exp(np.sum(- dt * np.array(sim_hw(gamma, sigma, theta, r0, num_steps, seed))))
    # dfs += math.exp(np.sum(sim))
ez = sim / num_sim
z_curva = math.exp(-zrate(t) * t)
print("ez: " + str(ez))
print("z_curva: " + str(z_curva))

num_steps: 66
ez: 0.9976732444021132
z_curva: 0.9976549482230134


In [ ]:
print("etasa: " + str(-100.0 * math.log(ez)/t))
print("tasa: " + str(-100.0 * math.log(z_curva)/t))

In [47]:
num_sim = 150000
t = .25
num_steps = int(t * 264)
print("num_steps: " + str(num_steps))
seed = None
df = 0
s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
    # print(len(sim))
    df += math.exp(-dt * np.sum(sim))
ez = df / num_sim
z_curva = math.exp(-zrate(t) * t)
print("ez: " + str(ez))
print("z_curva: " + str(z_curva))
print(t)

num_steps: 66
ez: 0.9976703273684974
z_curva: 0.9976549482230134
0.25
